In [3]:
%cd /content/drive/MyDrive/Tugas Akhir/TAnyaGusman/text_processing/location_extraction_src/

[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/Tugas Akhir/TAnyaGusman/text_processing/location_extraction_src/'
E:\data desktop\kuli ah\Term 7\TA\TA staging


In [6]:
%cd ..

E:\data desktop\kuli ah\Term 7\TA


In [35]:

%cd TAnyaGusman/text_processing/anotated_data/label_anotated_1


E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing\anotated_data\label_anotated_1


In [36]:
import pandas as pd

In [37]:
!dir

 Volume in drive E has no label.
 Volume Serial Number is 2DF2-6160

 Directory of E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing\anotated_data\label_anotated_1

15/11/2022  17:11    <DIR>          .
18/08/2022  14:09    <DIR>          ..
18/08/2022  14:09         1.335.135 angin_topan_text_label.xlsx
18/08/2022  14:09         1.254.635 banjir_text_label.xlsx
18/08/2022  14:09         1.251.190 erupsi_text_label.xlsx
18/08/2022  14:09         1.323.211 gempa_text_label.xlsx
18/08/2022  14:09         1.299.375 karhutla_text_label.xlsx
18/08/2022  14:09         1.102.726 kekeringan_text_label.xlsx
18/08/2022  14:09         1.304.566 longsor_text_label.xlsx
15/11/2022  17:11    <DIR>          split_index
08/11/2022  22:52                 0 stratified.py
18/08/2022  14:09         1.237.157 tsunami_text_label.xlsx
               9 File(s)     10.107.995 bytes
               3 Dir(s)  350.056.374.272 bytes free


In [38]:
!pip install openpyxl


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
atop = pd.read_excel('angin_topan_text_label.xlsx')

In [40]:
atop.head()

,id,source,date,title,text,label
0,angin_topan_0001,aceh_harianrakyat,"October 1, 2018",Puting Beliung Rusak Sembilan Rumah,Puting beliaung kembali terjadi di Bener Meria...,Y
1,angin_topan_0002,aceh_harianrakyat,"November 11, 2016",Pencarian Nelayan Simeulue Dihentikan,"Setelah sepekan operasi, pencarian dua nelayan...",Y
2,angin_topan_0003,aceh_harianrakyat,"January 4, 2018",Pohon Tumbang Timpa Warga Korban Meninggal Dal...,Seorang pengendara motor dilaporkan mengalami ...,Y
3,angin_topan_0004,aceh_harianrakyat,"February 11, 2020",Ulama Tetap Menjadi Referensi KPA/PA,Ulama tetap menjadi panutan dan petunjuk dalam...,N
4,angin_topan_0005,aceh_harianrakyat,"April 10, 2017",Empat Rumah Warga Rusak Dihantam Puting Beliung,Sebanyak empat rumah warga Desa Amaiteng Mulia...,Y


In [41]:
len(list(atop['source'].unique()))

63

## List of function to get instance data with positive or negative label

In [42]:
atop.head()
import os
#untuk tiap source, pilih 1 secara random label yang -, begitu jg untuk yg +
def get_test_negative_from_source(df, lst, dct):
    temp_df = df[["id", "label"]][df["label"]!="Y"][["id"]].sample()
    indeks = str(temp_df["id"])[-28:-24]
    if dct[indeks] == False:
        return get_test_negative_from_source(df, lst, dct)
    else:
        dct[indeks] = False
        lst.append(indeks)
        return lst, dct 

def get_test_positive_from_source(df, lst, dct):
    if len(df[df["label"]=="Y"])>=3:
        temp_df = df[["id", "label"]][df["label"]=="Y"][["id"]].sample()
        indeks = str(temp_df["id"])[-28:-24]
        if dct[indeks] == False:
            return get_test_positive_from_source(df, lst, dct)
        else:
            dct[indeks] = False
            lst.append(indeks)
            return lst, dct 
    else:
        return lst, dct 

def get_test_from_source(df, lst, dct):
    if len(df)<=3:
        for j in df["id"]:
            lst.append(j[-4:])
            dct[j[-4:]] = False 
        return lst, dct
        
    if len(df[df["label"]=="Y"])>=3:
        lst,dct = get_test_positive_from_source(df, lst, dct)
        lst, dct = get_test_negative_from_source(df, lst, dct)
        lst, dct = get_test_negative_from_source(df, lst, dct)
    else:
        lst, dct = get_test_negative_from_source(df, lst, dct)
        lst, dct = get_test_negative_from_source(df, lst, dct)
        lst, dct = get_test_negative_from_source(df, lst, dct)
    return lst, dct

def get_test_from_all_source(df):
    dct = {}
    for indek in df["id"]:
        dct[str(indek[-4:])] = True
    lst = []
    sources = list(df['source'].unique())
    for source in sources:
        lst,dct = get_test_from_source(df[df['source']==source], lst, dct)
    n = 0
    while len(lst)<200:
        n+=1
        if n%5==0:
            temp_df = df[["id", "label"]][df["label"]=="Y"][["id"]].sample()
            indeks = str(temp_df["id"])[-28:-24]
            if dct[indeks] != False:
                lst.append(indeks)
                dct[indeks] = False
            else:
                n-=1
        else:
            temp_df = df[["id", "label"]][df["label"]!="Y"][["id"]].sample()
            indeks = str(temp_df["id"])[-28:-24]
            if dct[indeks] != False:
                lst.append(indeks)
                dct[indeks] = False
            else:
                n-=1
    #print(f"panjang dari data test yang didapat sebanyak {len(lst)}")
    return lst,dct

def get_val_from_source(df, lst, dct):
    if len(df)<=4:
        for j in df["id"]:
            if dct[j[-4:]]:
                lst.append(j[-4:])
                dct[j[-4:]] = False 
        return lst, dct
        
    if len(df[df["label"]=="Y"])>=4:
        lst,dct = get_test_positive_from_source(df, lst, dct)
        lst, dct = get_test_negative_from_source(df, lst, dct)
    else:
        lst, dct = get_test_negative_from_source(df, lst, dct)
    return lst, dct

def get_val_from_all_source(df,dct):
    lst = []
    sources = list(df['source'].unique())
    for source in sources:
        lst,dct = get_val_from_source(df[df['source']==source], lst, dct)
    n = 0
    #print(f"Panjang dev saat ini {len(lst)}")
    while len(lst)<100:
        n+=1
        if n%7==0:
            temp_df = df[["id", "label"]][df["label"]=="Y"][["id"]].sample()
            indeks = str(temp_df["id"])[-28:-24]
            if dct[indeks] != False:
                lst.append(indeks)
                dct[indeks] = False
            else:
                n-=1
        else:
            temp_df = df[["id", "label"]][df["label"]!="Y"][["id"]].sample()
            indeks = str(temp_df["id"])[-28:-24]
            if dct[indeks] != False:
                lst.append(indeks)
                dct[indeks] = False
            else:
                n-=1
    #print(f"panjang dari data validation yang didapat sebanyak {len(lst)}")
    return lst, dct

def get_all_test_and_val(df):
    lst_test, dct = get_test_from_all_source(df)
    lst_dev, dct = get_val_from_all_source(df, dct)
    lst_train = [idx for idx in dct.keys() if dct[idx]]
    first =    set(lst_test).intersection(set(lst_dev))
    second = set(lst_test).intersection(set(lst_train))
    third = set(lst_train).intersection(set(lst_dev))
    if not (first and second and third):
        print(f"Telah didapatkan data train, dev, dan test sebanyak {len(lst_train)}, {len(lst_dev)}, dan {len(lst_test)}")
    else:
        print(f"Terdapat irisan antara train, dev, atau test ")
    return lst_train, lst_dev, lst_test

def create_splitting_file_index(lst, bencana_name, part_split):
    name_of_file = f"{bencana_name}_{ part_split }_index.txt"
    cwd = os.getcwd()
    parent_dir = os.path.dirname(os.path.dirname(cwd))
    save_path = os.path.join(parent_dir, "classification_split_index")
    try:
        os.mkdir(save_path)
    except:
        pass
    save_path = os.path.join(save_path, bencana_name)
    #print(save_path)
    try:
        os.mkdir(save_path)
    except:
        pass
    
    with open(os.path.join(save_path,name_of_file), 'w') as fp:
        pass
    
    print(lst)
    fd = os.open(os.path.join(save_path,name_of_file), os.O_RDWR)

    # String to be written
    s = str(lst)

    # Convert the string to bytes
    line = str.encode(s)

    numBytes = os.write(fd, line)
    os.close(fd)

def save_file_index(lst, bencana_name):
    create_splitting_file_index(lst[0], bencana_name, "train")
    create_splitting_file_index(lst[1], bencana_name, "dev")
    create_splitting_file_index(lst[2], bencana_name, "test")
def create_all_file_index():
    daftar_bencana = ["angin_topan",
                        "banjir",
                        "erupsi",
                        "gempa",
                        "karhutla",
                        "kekeringan",
                        "longsor",
                        "tsunami"
                    ]
    for bencana in daftar_bencana:
        df = pd.read_excel(f'{bencana}_text_label.xlsx')
        print(f"Bencana {bencana} akan di-split")
        save_file_index(list(get_all_test_and_val(df)), bencana)
        print(f"\n\nBERHASIL!!!, \n\nBencana {bencana} berhasil di-split")
        

In [43]:
create_all_file_index()

Bencana angin_topan akan di-split
Telah didapatkan data train, dev, dan test sebanyak 910, 100, dan 200
['0002', '0003', '0004', '0006', '0007', '0008', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0027', '0028', '0031', '0032', '0033', '0034', '0036', '0037', '0038', '0039', '0044', '0046', '0047', '0048', '0049', '0050', '0052', '0053', '0055', '0056', '0058', '0059', '0060', '0062', '0063', '0064', '0065', '0067', '0068', '0070', '0071', '0072', '0073', '0074', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0089', '0090', '0091', '0092', '0095', '0096', '0098', '0099', '0100', '0101', '0103', '0104', '0105', '0106', '0108', '0109', '0112', '0113', '0114', '0116', '0117', '0118', '0119', '0120', '0121', '0123', '0124', '0125', '0126', '0127', '0128', '0129', '0130', '0131', '0134', '0135', '0136', '0138', '0139', '0140', '0141', '0145', '0146', '0148', '0151', '0152', '0153', '0154',

Bencana banjir akan di-split
Telah didapatkan data train, dev, dan test sebanyak 936, 108, dan 200
['0001', '0002', '0003', '0005', '0006', '0008', '0010', '0011', '0012', '0014', '0016', '0018', '0019', '0020', '0022', '0023', '0024', '0025', '0026', '0028', '0029', '0030', '0033', '0034', '0036', '0037', '0038', '0039', '0040', '0044', '0045', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060', '0063', '0065', '0066', '0068', '0070', '0071', '0075', '0076', '0077', '0078', '0079', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '0090', '0091', '0092', '0094', '0095', '0098', '0099', '0100', '0102', '0104', '0105', '0106', '0107', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0118', '0120', '0121', '0122', '0123', '0124', '0125', '0127', '0128', '0129', '0130', '0133', '0135', '0136', '0138', '0139', '0140', '0141', '0142', '0143', '0144', '0146', '0147', '0148', '0149', '0150', '0151', '015

Telah didapatkan data train, dev, dan test sebanyak 790, 100, dan 200
['0001', '0002', '0003', '0004', '0006', '0007', '0008', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0019', '0021', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0033', '0035', '0036', '0038', '0039', '0040', '0044', '0045', '0047', '0048', '0050', '0051', '0052', '0054', '0057', '0058', '0059', '0061', '0062', '0063', '0064', '0065', '0066', '0068', '0069', '0071', '0073', '0074', '0075', '0077', '0078', '0080', '0081', '0082', '0083', '0085', '0087', '0088', '0089', '0090', '0093', '0094', '0095', '0096', '0097', '0098', '0100', '0101', '0103', '0104', '0105', '0106', '0107', '0109', '0110', '0111', '0112', '0113', '0115', '0116', '0118', '0120', '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0129', '0132', '0133', '0134', '0135', '0136', '0137', '0138', '0139', '0140', '0141', '0142', '0144', '0145', '0148', '0149', '0150', '0153', '0154', '0155', '0156', '0157', '0158', '

Bencana gempa akan di-split
Telah didapatkan data train, dev, dan test sebanyak 917, 100, dan 200
['0001', '0002', '0003', '0004', '0005', '0007', '0009', '0012', '0013', '0014', '0015', '0016', '0018', '0019', '0020', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0030', '0031', '0032', '0035', '0036', '0037', '0038', '0044', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0055', '0056', '0057', '0058', '0059', '0060', '0063', '0064', '0065', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0076', '0077', '0078', '0080', '0081', '0084', '0085', '0086', '0087', '0088', '0090', '0091', '0093', '0094', '0096', '0097', '0098', '0099', '0100', '0102', '0103', '0105', '0106', '0107', '0111', '0112', '0115', '0116', '0117', '0118', '0119', '0120', '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0129', '0130', '0131', '0132', '0133', '0135', '0136', '0138', '0139', '0140', '0141', '0142', '0143', '0146', '0147', '0148', '0149', '0152', '0154

Telah didapatkan data train, dev, dan test sebanyak 842, 107, dan 200
['0001', '0003', '0005', '0006', '0008', '0010', '0011', '0012', '0013', '0014', '0015', '0017', '0018', '0019', '0020', '0022', '0023', '0024', '0025', '0026', '0028', '0030', '0032', '0033', '0034', '0035', '0036', '0038', '0039', '0040', '0043', '0044', '0045', '0047', '0048', '0049', '0051', '0054', '0055', '0056', '0057', '0058', '0059', '0060', '0061', '0064', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0077', '0078', '0081', '0082', '0083', '0084', '0085', '0088', '0089', '0090', '0091', '0092', '0093', '0095', '0096', '0097', '0098', '0102', '0103', '0104', '0106', '0107', '0108', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0119', '0120', '0122', '0123', '0124', '0125', '0126', '0128', '0129', '0131', '0132', '0133', '0134', '0135', '0138', '0139', '0140', '0141', '0142', '0143', '0144', '0145', '0146', '0147', '0150', '0152', '0153', '0154', '0156', '0158', '0159', '

Telah didapatkan data train, dev, dan test sebanyak 798, 100, dan 200
['0002', '0003', '0004', '0005', '0007', '0008', '0010', '0013', '0014', '0015', '0016', '0017', '0022', '0024', '0025', '0026', '0027', '0029', '0030', '0031', '0032', '0033', '0035', '0037', '0039', '0040', '0041', '0043', '0044', '0045', '0046', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0056', '0058', '0059', '0060', '0061', '0062', '0064', '0065', '0066', '0068', '0069', '0070', '0072', '0073', '0074', '0075', '0077', '0078', '0079', '0081', '0082', '0083', '0084', '0085', '0086', '0088', '0089', '0090', '0091', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0103', '0106', '0108', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0119', '0120', '0121', '0124', '0125', '0126', '0127', '0128', '0131', '0133', '0134', '0135', '0136', '0138', '0139', '0140', '0141', '0142', '0143', '0146', '0147', '0148', '0149', '0150', '0153', '0154', '0155', '0156', '

Bencana longsor akan di-split
Telah didapatkan data train, dev, dan test sebanyak 915, 105, dan 200
['0003', '0005', '0006', '0007', '0008', '0009', '0011', '0012', '0013', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0026', '0028', '0029', '0030', '0032', '0035', '0036', '0037', '0038', '0039', '0040', '0045', '0046', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0057', '0059', '0060', '0062', '0063', '0064', '0066', '0067', '0068', '0069', '0071', '0073', '0074', '0075', '0076', '0077', '0078', '0079', '0081', '0082', '0086', '0087', '0088', '0089', '0090', '0091', '0092', '0093', '0095', '0097', '0098', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0109', '0111', '0113', '0114', '0115', '0116', '0117', '0118', '0119', '0120', '0121', '0122', '0124', '0125', '0126', '0128', '0130', '0131', '0132', '0133', '0134', '0135', '0137', '0138', '0140', '0142', '0143', '0144', '0146', '0147', '0148', '0149', '0150', '0151', '01

Bencana tsunami akan di-split
Telah didapatkan data train, dev, dan test sebanyak 865, 100, dan 200
['0001', '0002', '0003', '0005', '0006', '0007', '0008', '0009', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0021', '0023', '0027', '0028', '0029', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0040', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0055', '0058', '0059', '0060', '0061', '0062', '0063', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0077', '0078', '0080', '0082', '0083', '0084', '0086', '0087', '0088', '0089', '0090', '0091', '0092', '0093', '0094', '0095', '0096', '0097', '0099', '0102', '0104', '0105', '0106', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0119', '0120', '0121', '0124', '0125', '0126', '0127', '0128', '0129', '0130', '0132', '0133', '0135', '0136', '0137', '0138', '0139', '0140', '0141', '0144', '0145', '0146', '0148', '0149', '01

In [20]:
atop.head()

,id,source,date,title,text,label
0,angin_topan_0001,aceh_harianrakyat,"October 1, 2018",Puting Beliung Rusak Sembilan Rumah,Puting beliaung kembali terjadi di Bener Meria...,Y
1,angin_topan_0002,aceh_harianrakyat,"November 11, 2016",Pencarian Nelayan Simeulue Dihentikan,"Setelah sepekan operasi, pencarian dua nelayan...",Y
2,angin_topan_0003,aceh_harianrakyat,"January 4, 2018",Pohon Tumbang Timpa Warga Korban Meninggal Dal...,Seorang pengendara motor dilaporkan mengalami ...,Y
3,angin_topan_0004,aceh_harianrakyat,"February 11, 2020",Ulama Tetap Menjadi Referensi KPA/PA,Ulama tetap menjadi panutan dan petunjuk dalam...,N
4,angin_topan_0005,aceh_harianrakyat,"April 10, 2017",Empat Rumah Warga Rusak Dihantam Puting Beliung,Sebanyak empat rumah warga Desa Amaiteng Mulia...,Y


In [21]:
!dir

 Volume in drive E has no label.
 Volume Serial Number is 2DF2-6160

 Directory of E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing\anotated_data\label_anotated_1

15/11/2022  17:11    <DIR>          .
18/08/2022  14:09    <DIR>          ..
18/08/2022  14:09         1.335.135 angin_topan_text_label.xlsx
18/08/2022  14:09         1.254.635 banjir_text_label.xlsx
18/08/2022  14:09         1.251.190 erupsi_text_label.xlsx
18/08/2022  14:09         1.323.211 gempa_text_label.xlsx
18/08/2022  14:09         1.299.375 karhutla_text_label.xlsx
18/08/2022  14:09         1.102.726 kekeringan_text_label.xlsx
18/08/2022  14:09         1.304.566 longsor_text_label.xlsx
15/11/2022  17:11    <DIR>          split_index
08/11/2022  22:52                 0 stratified.py
18/08/2022  14:09         1.237.157 tsunami_text_label.xlsx
               9 File(s)     10.107.995 bytes
               3 Dir(s)  350.056.378.368 bytes free


In [44]:
!dir
print(os.path.dirname(os.path.dirname(os.getcwd())))
%cd ..
%cd ..
%cd classification_staging\preproc_text
print(os.path.dirname(os.path.dirname(os.getcwd())))

 Volume in drive E has no label.
 Volume Serial Number is 2DF2-6160

 Directory of E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing\anotated_data\label_anotated_1

15/11/2022  17:11    <DIR>          .
18/08/2022  14:09    <DIR>          ..
18/08/2022  14:09         1.335.135 angin_topan_text_label.xlsx
18/08/2022  14:09         1.254.635 banjir_text_label.xlsx
18/08/2022  14:09         1.251.190 erupsi_text_label.xlsx
18/08/2022  14:09         1.323.211 gempa_text_label.xlsx
18/08/2022  14:09         1.299.375 karhutla_text_label.xlsx
18/08/2022  14:09         1.102.726 kekeringan_text_label.xlsx
18/08/2022  14:09         1.304.566 longsor_text_label.xlsx
15/11/2022  17:11    <DIR>          split_index
08/11/2022  22:52                 0 stratified.py
18/08/2022  14:09         1.237.157 tsunami_text_label.xlsx
               9 File(s)     10.107.995 bytes
               3 Dir(s)  350.056.374.272 bytes free
E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing
E:\d

## save splitted file

In [48]:
def create_splitted_csv_file(df, bencana_name, part_split):
    name_of_file = f"{bencana_name}_{ part_split }.csv"
    cwd = os.getcwd()
    parent_dir = os.path.dirname(os.path.dirname(cwd))
    save_path = os.path.join(parent_dir, "classification_splited_csv_file")
    
    try:
        os.mkdir(save_path)
    except:
        pass
    
    save_path = os.path.join(save_path, bencana_name)
    #print(save_path)
    try:
        os.mkdir(save_path)
    except:
        pass
    
    df.to_csv(os.path.join(save_path,name_of_file))
    
    
def create_all_file_csv_splitted():
    daftar_bencana = ["angin_topan",
                        "banjir",
                        "erupsi",
                        "gempa",
                        "karhutla",
                        "kekeringan",
                        "longsor",
                        "tsunami"
                    ]
    for bencana in daftar_bencana:
        for part_split in ["train", "dev", "test"]:
            df = get_df_from_index(bencana, part_split)
            create_splitted_csv_file(df, bencana, part_split)
        

In [49]:
create_all_file_csv_splitted()

In [51]:
%cd ..
%cd ..

E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing\classification_staging
E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing


In [52]:
%cd classification_splited_csv_file

E:\data desktop\kuli ah\Term 7\TA\TAnyaGusman\text_processing\classification_splited_csv_file


In [53]:
daftar_bencana = ['angin_topan', 'banjir','erupsi', 'gempa', 'karhutla', 'kekeringan', 'longsor', 'tsunami']
print('Statistik data persebaran label positif dan negatif')
for bencana in daftar_bencana:
    print("###########")
    print(f"Pada bencana {bencana}, terdapat")
    print("train:")
    data_train = pd.read_csv(f'./{bencana}/{bencana}_train.csv')
    print(f"{len(data_train[data_train['label']=='Y'])} data positif")
    print(f"{len(data_train[data_train['label']!='Y'])} data negatif")
    print(f"perbandingannya {1}:{len(data_train[data_train['label']!='Y'])/len(data_train[data_train['label']=='Y'])}")
    print("-------------")
    
    print("dev:")
    data_dev = pd.read_csv(f'./{bencana}/{bencana}_dev.csv')
    print(f"{len(data_dev[data_dev['label']=='Y'])} data positif")
    print(f"{len(data_dev[data_dev['label']!='Y'])} data negatif")
    print(f"perbandingannya {1}:{len(data_dev[data_dev['label']!='Y'])/len(data_dev[data_dev['label']=='Y'])}")
    print("-------------")
    
    print("test:")
    data_test = pd.read_csv(f'./{bencana}/{bencana}_test.csv')
    print(f"{len(data_test[data_test['label']=='Y'])} data positif")
    print(f"{len(data_test[data_test['label']!='Y'])} data negatif")
    print(f"perbandingannya {1}:{len(data_test[data_test['label']!='Y'])/len(data_test[data_test['label']=='Y'])}")
    print("-------------")
    print('\n')

Statistik data persebaran label positif dan negatif
###########
Pada bencana angin_topan, terdapat
train:
159 data positif
751 data negatif
perbandingannya 1:4.723270440251572
-------------
dev:
32 data positif
68 data negatif
perbandingannya 1:2.125
-------------
test:
46 data positif
154 data negatif
perbandingannya 1:3.347826086956522
-------------


###########
Pada bencana banjir, terdapat
train:
221 data positif
715 data negatif
perbandingannya 1:3.235294117647059
-------------
dev:
45 data positif
63 data negatif
perbandingannya 1:1.4
-------------
test:
54 data positif
146 data negatif
perbandingannya 1:2.7037037037037037
-------------


###########
Pada bencana erupsi, terdapat
train:
95 data positif
695 data negatif
perbandingannya 1:7.315789473684211
-------------
dev:
18 data positif
82 data negatif
perbandingannya 1:4.555555555555555
-------------
test:
34 data positif
166 data negatif
perbandingannya 1:4.882352941176471
-------------


###########
Pada bencana gempa, terd